<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/Unet_train_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BASICS 
import os
import numpy as np
import torch


# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [ ]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

Cloning into 'lung_covid'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 133 (delta 13), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (133/133), 1.11 MiB | 5.80 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.15G/1.15G [00:48<00:00, 38.1MB/s]
100% 1.15G/1.15G [00:48<00:00, 25.5MB/s]


In [ ]:
# The last part to hide output bacause it is more then 30000 lines
!unzip "/content/covidqu.zip" -d "/content/archive/" &> /dev/null

In [ ]:
# running  .ipynb scripts to call their functions
%run lung_covid/functions.ipynb
%run lung_covid/unet.ipynb

# !cp -avr lung_covid/model/ .
# from model.unet import UNet, PretrainedUNet

In [ ]:
# The Calling the validaton dataset
val=Lung_Dataset_RGB(split = "Val",class_folder='covid',shuffle=True,sample_size=700)# The length of the validation dataset to calculate the average value of the loss and accuracy
# The size of the dataset for calculate average datas
val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset_RGB(split = "Train",class_folder='covid',shuffle=True,sample_size=4000)
tra_size=tra.__len__()
# # The transform functions to conver the PIL images to the tensors


# The device looks for GPU if cannot find goes with CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# The model is called by this function from the model.py file 
net = Unet_Res().to(device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
batch_size=30
# loading images and masks for mini batches
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [ ]:
# The stiation of the GPU
!nvidia-smi

Wed Jun  8 07:20:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    31W / 250W |   1185MiB / 16280MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# The number of the epoch 
epoch_number=50
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        # The negative takes the masks and doubles the channels with inverse (0 to 1) shape  
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=binary_acc( masks_pred, true_masks).item()
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=binary_acc( masks_pred, true_masks).item()
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

Epoch : 1 / 50
Validation Loss:v 0.2482441944735391
Validation Acc: 0.9348357234682356
Train Acc: 0.7158027611704855
Train  Loss   :t 0.46739648907673204
Epoch : 2 / 50
Validation Loss:v 0.12554005309939384
Validation Acc: 0.9457283096654076
Train Acc: 0.9636491012675568
Train  Loss   :t 0.1588558509961858
Epoch : 3 / 50
Validation Loss:v 0.09872803145221302
Validation Acc: 0.9497457095554898
Train Acc: 0.9735532304823654
Train  Loss   :t 0.08324125115140121
Epoch : 4 / 50
Validation Loss:v 0.1349976472556591
Validation Acc: 0.9373030722141267
Train Acc: 0.9777687982428227
Train  Loss   :t 0.05852176587980704
Epoch : 5 / 50
Validation Loss:v 0.07665880817387785
Validation Acc: 0.953905805519649
Train Acc: 0.980572510078997
Train  Loss   :t 0.04625359450191131
Epoch : 6 / 50
Validation Loss:v 0.0747901452439172
Validation Acc: 0.9560598518167224
Train Acc: 0.9831793424013858
Train  Loss   :t 0.03741582944517548
Epoch : 7 / 50
Validation Loss:v 0.07525913012879236
Validation Acc: 0.95790

In [ ]:
import copy
# Save Model to the drive folder 
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':epoch_number,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict()
     
},os.path.join(os.getcwd(),'UNet_covid_{}_epoch.pt'.format(epoch_number)))

In [ ]:
# Save loss and acc information to the txt 
textfile = open("UNet_covid_{}_epo.txt".format(epoch_number), "w")
# The form of the saved values are important when we used them to plot
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()
